In [1]:
from sagemaker import get_execution_role

In [2]:
custom_code_upload_location = 's3://sagemaker-02122018/customcode/tensorflow_iris'

In [3]:
model_artifacts_location = 's3://sagemaker-02122018/artifacts'


import boto3
import os

bucket = 'sagemaker-02122018'
prefix = 'artifacts/tensorflow-training-2020-09-26-15-51-06-130/model'

s3_bucket_resource = boto3.resource('s3').Bucket(bucket)


output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))



In [4]:
role = get_execution_role()

In [5]:
!cat "iris_dnn_1.py" 

from __future__ import absolute_import

import numpy as np
import os
import tensorflow as tf

INPUT_TENSOR_NAME = 'inputs'
input_tensor_name = 'inputs'

def estimator_fn(run_config, params): #Function 1 Estimator 
    feature_columns = [tf.feature_column.numeric_column(INPUT_TENSOR_NAME, shape=[4])] #shape represents the four elements of the data set 
    return tf.estimator.DNNClassifier(feature_columns=feature_columns,#s
                                      hidden_units=[10, 20, 10], #Deep Neural Network Classifier
                                      n_classes=3, #Three output classes because we have three species of flowers
                                      config=run_config)


def serving_input_fn(params):  #Function 2 Once the model is deployed how model is going to get the inputs 
    feature_spec = {INPUT_TENSOR_NAME: tf.placeholder(tf.float32,[None,4],name=None)}
    return tf.estimator.export.ServingInputReceiver(feature_spec,feature_spec)


def train_input_fn(training_

In [6]:
from sagemaker.tensorflow import TensorFlow

In [7]:
iris_estimator = TensorFlow(entry_point='iris_dnn_1.py',  #Creating Estimator,etreing python scrypt
                            role=role, #IAM Role
                            framework_version='1.15.0',
                            output_path=model_artifacts_location,
                            code_location=custom_code_upload_location,
                            train_instance_count=1, #no of instences
                            train_instance_type='ml.c4.xlarge',
                            training_steps=1000,  
                            evaluation_steps=100)


2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.
Legacy mode is deprecated in versions 1.13 and higher. Using script mode instead. Legacy mode and its training parameters will be deprecated in SageMaker Python SDK v2. Please use TF 1.13 or higher and script mode.


In [8]:
%%time
import boto3

region = boto3.Session().region_name
train_data_location = 's3://sagemaker-sample-data-{}/tensorflow/iris'.format(region)

iris_estimator.fit(train_data_location)


'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-09-26 15:51:06 Starting - Starting the training job...
2020-09-26 15:51:08 Starting - Launching requested ML instances......
2020-09-26 15:52:32 Starting - Preparing the instances for training..................
2020-09-26 15:55:21 Downloading - Downloading input data
2020-09-26 15:55:21 Training - Downloading the training image..WARNING:tensorflow:From /usr/local/lib/python2.7/dist-packages/tensorflow_core/__init__.py:1467: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.

2020-09-26 15:55:46,601 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-09-26 15:55:46,608 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-09-26 15:55:47,259 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-09-26 15:55:47,275 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-09-26 15:55:47,294 sagemaker-containers INFO     No

training artifacts will be uploaded to: s3://sagemaker-02122018/artifacts/tensorflow-training-2020-09-26-15-51-06-130/model/output


In [9]:
# %%time

# iris_predictor = iris_estimator.deploy(initial_instance_count=1,
#                                        instance_type='ml.c4.xlarge')


In [10]:
# iris_predictor.predict([7.0, 3.2, 4.7, 1.4])

In [11]:
output_path = '/'.join(iris_estimator.output_path.split('/')[:-1])
optimized_estimator = iris_estimator.compile_model(target_instance_family='rasp3b', 
                              input_shape={'data':[1, 4]},  # Batch size 1, 3 channels, 224x224 Images.
                              output_path=output_path,
                              framework='tensorflow', framework_version='1.15.0')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


?*

UnexpectedStatusException: Error for Compilation job compilation-tensorflow-training-2020-09-26-15-56-18-638: Failed. Reason: ClientError: InputConfiguration: S3 object does not exist. Bucket: sagemaker-02122018, Key: artifacts/tensorflow-training-2020-09-26-15-51-06-130/output/model.tar.gz